In [2]:
##########################################################################################
# Abraham Tishelman-Charny                                                               #
# 3 March 2021                                                                           #
#                                                                                        #
# The purpose of this notebook is to validate Resonant HH UL gridpacks. This can         #
# also be used to study any MadGraph 5 gridpacks.                                        #
##########################################################################################

# Thanks to:
# https://pydoc.net/pylhe/0.0.2/pylhe/ ##-- pylhe 
# https://github.com/scikit-hep/pylhe/blob/master/examples/zpeak.ipynb

In [2]:
##-- Imports 
import pylhe 
import ROOT
import math
from matplotlib import pyplot as plt 
import numpy as np 
import os 
import pandas as pd 
import pickle 
from ROOT import TLorentzVector

In [41]:
##-- Get LHE Files assuming directory tree structure: LHE_Files/<gridpack_Type>/<files>
# gridpack_Type = "GF_Spin-0"
# gridpack_Type = "GF_Spin-2"
# gridpack_Type = "VBF_Spin-0"
gridpack_Type = "VBF_Spin-2"
#files = ["test/cmsgrid_final.lhe"]
os.system("pwd")
files = ["lhe_production/LHE_Files/%s/%s"%(gridpack_Type,file) for file in os.listdir('./lhe_production/LHE_Files/%s'%(gridpack_Type)) if file.endswith('.lhe')] 
for file in files:
    print("Found File:",file)
print("N files:",len(files))

Found File: lhe_production/LHE_Files/VBF_Spin-2/VBFToBulkGravitonToHH_M1000_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe
Found File: lhe_production/LHE_Files/VBF_Spin-2/VBFToBulkGravitonToHH_M1250_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe
Found File: lhe_production/LHE_Files/VBF_Spin-2/VBFToBulkGravitonToHH_M1500_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe
Found File: lhe_production/LHE_Files/VBF_Spin-2/VBFToBulkGravitonToHH_M1750_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe
Found File: lhe_production/LHE_Files/VBF_Spin-2/VBFToBulkGravitonToHH_M2000_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe
Found File: lhe_production/LHE_Files/VBF_Spin-2/VBFToBulkGravitonToHH_M2500_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe
Found File: lhe_production/LHE_Files/VBF_Spin-2/VBFToBulkGravitonToHH_M250_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe
Found File: lhe_production/LHE_Files/VBF_Spin-2/VBFToBulkGravitonToHH_M260_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe
Found File: lhe_production/LHE_Files/VBF_S

In [46]:
##-- Create Pandas dataframe for each file 

##-- Parameters 
##################
debug = 0      
maxEvents = 1000
lessPoints = 0
##################

##-- Element in lhe file name which has mass information 
massGPtypeDict = {
    "GF_Spin-0" : 3,
    "GF_Spin-2" : 5,
    "VBF_Spin-0" : 1,
    "VBF_Spin-2" : 1
}
massElement = massGPtypeDict[gridpack_Type]

##-- Functions for Event Loop
def GetPt(p):
    px = p.px
    py = p.py 
    pt = np.sqrt(px**2 + py**2)
    return pt

def GetDR(eta1, eta2, phi1, phi2):
    DR = np.sqrt((eta1 - eta2)**2 + (phi1 - phi2)**2)
    return DR

##-- For each LHE file (one per gridpack)
for i_file, file in enumerate(files):

    ##-- File Printout
    print("On file:",file)
    mass = file.split('/')[-1].split('_')[massElement]
    mass = mass.replace("M", "")
    print("mass:", mass)  
    
    if(lessPoints):
        if(mass != "250" and mass != "3000"): continue 
    
    ##-- Prepare dictionary of lists for pandas dataframe 
    d = {}
    
    ##-- Define particles based on gridpack type 
    particleDictionary = {
        "GF_Spin-0" : ["X","Lead_H","Sublead_H","Lead_g","Sublead_g"], 
        "GF_Spin-2" : ["X","Lead_H","Sublead_H","Lead_g","Sublead_g"],
        "VBF_Spin-0" : ["X","Lead_H","Sublead_H","Lead_Incoming_q","Sublead_Incoming_q", "Lead_Outgoing_q", "Sublead_Outgoing_q"], 
        "VBF_Spin-2" : ["X","Lead_H","Sublead_H","Lead_Incoming_q","Sublead_Incoming_q", "Lead_Outgoing_q", "Sublead_Outgoing_q"]
    }
    
    ##-- Define particles to determine lead / sublead for 
    doubleParticleDict = {
        "GF_Spin-0" : ["H", "g"], 
        "GF_Spin-2" : ["H", "g"], 
        "VBF_Spin-0" : ["H", "Incoming_q", "Outgoing_q"], 
        "VBF_Spin-2" : ["H", "Incoming_q", "Outgoing_q"]
    }    
    
    particle_Names = particleDictionary[gridpack_Type]
    doubleParticles = doubleParticleDict[gridpack_Type]
    
    ##-- Define variables to save for each particle
    variables = ["px", "py", "pz", "e", "m", "spin", "status", "lifetime", "pt"]
    ##-- possible variables: ['id','status','color1','color2','px','py','pz','e','m','lifetime','spin']  
    
    ##-- Initialize dictionary lists 
    for p in particle_Names:
        for v in variables:
            columnName = "%s_%s"%(p,v)
            d[columnName] = [] 
    
    ##-- Save both Higgs' eta and phi 
    for H_particle in ["Lead_H", "Sublead_H", "X"]:
        for H_var in ["eta", "phi"]:
            exec("d['%s_%s'] = []"%(H_particle, H_var))

    ##-- For both GF and VBF: Initialize lists to save event by event pdgIds, DR(H, H), Deta(HH), Dphi(HH)
    d["pdgIds"] = []
    d["DR_HH"] = []
    d["HH_Deta"] = []
    d["HH_Dphi"] = []
    
    ##-- VBF Case: Initialize lists to save event by event eta and phi of incoming / outgoing quarks 
    if("VBF" in gridpack_Type):
        for q_particle in ["Lead_Outgoing_q", "Sublead_Outgoing_q", "Lead_Incoming_q", "Sublead_Incoming_q"]:
            for q_var in ["eta", "phi"]:
                exec("d['%s_%s'] = []"%(q_particle, q_var))   
               
        ##-- Save DR(outgoing_quark_1, outgoing_quark_2), DEta and DPhi for VBF case 
        d['DR_OutqOutq'] = []
        d['OutqOutq_Deta'] = []
        d['OutqOutq_Dphi'] = []
        
        ##-- Same for incoming quarks (probably not as interesting (?))
        d['DR_InqInq'] = []
        d['InqInq_Deta'] = []
        d['InqInq_Dphi'] = []         
    
    ##-- For Each Event
    print("Looping Events ...")
    for ie, event in enumerate(pylhe.readLHE(file)):
        if(ie%1000 == 0): print("On Event %s"%(ie))
        if(ie == maxEvents): 
            print("STOPPING at %s events"%(maxEvents))
            break 
        particles = event.particles
        pdgIds = []
        foundFirstHiggs, foundFirstGluon = 0, 0  
        foundFirstIncomingQuark, foundFirstOutgoingQuark = 0, 0
        
        ##-- Get Particles
        for particle in particles:
            pdgId = particle.id
            pdgIds.append(pdgId)
            status = particle.status
            
            ##-- Lots of particle information if debugging
            if(debug): 
                print("")
                print("pdgId:",pdgId)
                print("mass:",particle.m)
                print("mother1:",particle.mother1)
                print("mother2:",particle.mother2)
                print("color1:",particle.color1)
                print("color2:",particle.color2)                
                print("status:",particle.status)
                print("lifetime:",particle.lifetime)  
                print("px:",particle.px)  
                print("py:",particle.py)  
                print("pz:",particle.pz)  
                print("spin:",particle.spin)
            
            ##-- Gluons (GF only)
            if(pdgId == 21 and ("GF" in gridpack_Type) and not foundFirstGluon):
                foundFirstGluon = 1 
                g1 = particle 
            elif(pdgId == 21 and ("GF" in gridpack_Type) and foundFirstGluon):
                g2 = particle  
                
            ##-- Incoming Quarks (VBF only)
            ##-- For some reason incoming VBF quarks are defined as pdgId == 21 in the LHE file. Guessing it's due to gridpack model import 
            elif(pdgId == 21 and status == -1.0 and ("VBF" in gridpack_Type) and not foundFirstIncomingQuark):
                foundFirstIncomingQuark = 1 
                Incoming_q1 = particle
            elif(pdgId == 21 and status == -1.0 and ("VBF" in gridpack_Type) and foundFirstIncomingQuark):
                Incoming_q2 = particle     
                
            ##-- Outgoing Quarks (VBF only)
            ##-- For some reason incoming VBF quarks are defined as pdgId == 21 in the LHE file. Guessing it's due to gridpack model import
            elif(pdgId == 21 and status == 1 and not foundFirstOutgoingQuark):
                foundFirstOutgoingQuark = 1 
                Outgoing_q1 = particle
            elif(pdgId == 21 and status == 1 and foundFirstOutgoingQuark):
                Outgoing_q2 = particle                 
                
            ##-- Intermediate vector bosons are exactly equal and opposite in px, py, pz to outgoing quarks (when there is no ISR and FSR from showering)
            ##-- therefore, information of outgoing quarks gives you information of intermediate vector bosons (in LHE case with no ISR and FSR)
            
            ##-- Spin-0 or Spin-2 Resonant Particle
            if(pdgId == 35 or pdgId == 39):
                X = particle 
                
            ##-- Higgs Bosons 
            if(pdgId == 25 and not foundFirstHiggs):
                foundFirstHiggs = 1 
                H1 = particle 
            elif(pdgId == 25 and foundFirstHiggs):
                H2 = particle 
                
        ##-- Determine Leading, Subleading particles based on pT, save variables 
        for p in doubleParticles:
            p1 = eval("%s1"%(p))
            p2 = eval("%s2"%(p))
            p1_pt = GetPt(p1)
            p2_pt = GetPt(p2)
            if(p1_pt > p2_pt):
                exec("Lead_%s = p1"%(p))
                exec("Sublead_%s = p2"%(p))
            elif(p2_pt > p1_pt):
                exec("Lead_%s = p2"%(p))
                exec("Sublead_%s = p1"%(p)) 
            else:
                exec("Lead_%s = p1"%(p))
                exec("Sublead_%s = p2"%(p))   
            
            ##-- Save variables 
            for v in variables:
                
                ##-- pt gets special treatment
                if(v == "pt"):
                    ##-- Leading variable expression, subleading variable expression
                    exec("d['Lead_%s_%s'].append(p1_pt)"%(p, v))
                    exec("d['Sublead_%s_%s'].append(p2_pt)"%(p, v))
                else:
                    ##-- Leading variable expression, subleading variable expression
                    lvExp = "Lead_%s.%s"%(p, v)
                    slvExp = "Sublead_%s.%s"%(p, v)
                    exec("Lead_%s_%s = %s"%(p, v, lvExp))
                    exec("Sublead_%s_%s = %s"%(p, v, slvExp))
                    exec("d['Lead_%s_%s'].append(Lead_%s_%s)"%(p, v, p, v))
                    exec("d['Sublead_%s_%s'].append(Sublead_%s_%s)"%(p, v, p, v))                    
                
        ##-- Resonant particle
        X_pt = GetPt(X)
        for v in variables:
            c = "X_%s"%(v)
            if(v == "pt"):
                X_pt = GetPt(X)
            else:
                exec("%s = X.%s"%(c, v))
            exec("d['%s'].append(%s)"%(c, c))
        
        # X eta/phi:
        tlv_X = TLorentzVector()
        tlv_X.SetPxPyPzE(X.px,X.py,X.pz,X.e)
        d['X_eta'].append(tlv_X.Eta())
        d['X_phi'].append(tlv_X.Phi())
        
        ##-- DeltaR between pairs of particles 
        for p_Name in doubleParticles:
            Lead_p_v = TLorentzVector() ##-- Lead particle vector
            Sublead_p_v = TLorentzVector() ##-- Subleading particle vector 
            
            ##-- Set pxpypzE of TLorentzVector in order to get Eta and Phi 
            for rank in ["Lead", "Sublead"]:
                for kinVar in ["px", "py", "pz", "e"]:
                    exec("%s_p_%s = %s_%s_%s"%(rank, kinVar, rank, p_Name, kinVar))

            Lead_p_v.SetPxPyPzE(Lead_p_px, Lead_p_py, Lead_p_pz, Lead_p_e)
            Sublead_p_v.SetPxPyPzE(Sublead_p_px, Sublead_p_py, Sublead_p_pz, Sublead_p_e)

            ##-- Get Eta and Phi 
            Lead_p_eta = Lead_p_v.Eta()
            Lead_p_phi = Lead_p_v.Phi()
            Sublead_p_eta = Sublead_p_v.Eta()
            Sublead_p_phi = Sublead_p_v.Phi()
            
            Deta = float(Lead_p_eta - Sublead_p_eta)
            Dphi = float(Lead_p_phi - Sublead_p_phi)

            for rank in ["Lead", "Sublead"]:
                for ang_var in ["eta", "phi"]:
                    exec("d['%s_%s_%s'].append(%s_p_%s)"%(rank, p_Name, ang_var, rank, ang_var))

            DR = GetDR(Lead_p_eta, Sublead_p_eta, Lead_p_phi, Sublead_p_phi)

            if(p_Name == "H"):
                d['DR_HH'].append(DR)
                d['HH_Deta'].append(Deta)
                d['HH_Dphi'].append(Dphi)
            elif(p_Name == "Outgoing_q"):
                d['DR_OutqOutq'].append(DR)
                d['OutqOutq_Deta'].append(Deta)
                d['OutqOutq_Dphi'].append(Dphi)  
            elif(p_Name == "Incoming_q"):
                d['DR_InqInq'].append(DR)
                d['InqInq_Deta'].append(Deta)
                d['InqInq_Dphi'].append(Dphi)                  
        
        ##-- Append event pdgIds
        d['pdgIds'].append(pdgIds)
        
    ##-- Useful debug is making sure length of each variable list is the same after looping events 
    if(debug):
        for p in particle_Names:
            for v in variables:
                columnName = "%s_%s"%(p,v)
                print("columnName:",columnName)
                print("len(%s):"%(columnName),len(d[columnName]))          
        
    ##-- After event loop, save dataframe
    df = pd.DataFrame(data = d)
    if(debug): print(df)
    pickle.dump(df, open( "lhe_production/Dataframes/%s/%s.p" % (gridpack_Type, mass), "wb" )) ##-- Pickle the dataframe 
#    pickle.dump(df, open( "%s_%s.p" % (gridpack_Type, mass), "wb" )) ##-- Pickle the dataframe           
print("DONE")


On file: lhe_production/LHE_Files/VBF_Spin-2/VBFToBulkGravitonToHH_M1000_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe
mass: 1000
Looping Events ...
On Event 0
On Event 1000
STOPPING at 1000 events
On file: lhe_production/LHE_Files/VBF_Spin-2/VBFToBulkGravitonToHH_M1250_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe
mass: 1250
Looping Events ...
On Event 0
On Event 1000
STOPPING at 1000 events
On file: lhe_production/LHE_Files/VBF_Spin-2/VBFToBulkGravitonToHH_M1500_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe
mass: 1500
Looping Events ...
On Event 0
On Event 1000
STOPPING at 1000 events
On file: lhe_production/LHE_Files/VBF_Spin-2/VBFToBulkGravitonToHH_M1750_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe
mass: 1750
Looping Events ...
On Event 0
On Event 1000
STOPPING at 1000 events
On file: lhe_production/LHE_Files/VBF_Spin-2/VBFToBulkGravitonToHH_M2000_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe
mass: 2000
Looping Events ...
On Event 0
On Event 1000
STOPPING at 1000 events
On file: lhe_product

In [47]:
##-- To check a dataframe 
df = pickle.load( open( "lhe_production/Dataframes/VBF_Spin-2/1000.p", "rb" ) )
#df = pickle.load( open( "./VBF_Spin-2_final.lhe.p", "rb" ) )
df

,X_px,X_py,X_pz,X_e,X_m,X_spin,X_status,X_lifetime,X_pt,Lead_H_px,...,Lead_Incoming_q_eta,Lead_Incoming_q_phi,Sublead_Incoming_q_eta,Sublead_Incoming_q_phi,DR_OutqOutq,OutqOutq_Deta,OutqOutq_Dphi,DR_InqInq,InqInq_Deta,InqInq_Dphi
0,10.167490,-58.699730,744.274747,1247.995144,999.998973,0.0,2.0,0.0,59.573788,202.343990,...,1.000000e+11,0.0,-1.000000e+11,0.0,1.908923,-0.416426,1.862949,2.000000e+11,2.000000e+11,0.0
1,118.960766,38.383457,339.705838,1063.495321,999.998644,0.0,2.0,0.0,124.999814,239.011968,...,1.000000e+11,0.0,-1.000000e+11,0.0,1.908923,-0.416426,1.862949,2.000000e+11,2.000000e+11,0.0
2,-149.813228,149.589396,427.312085,1107.888981,1000.000693,0.0,2.0,0.0,211.709685,-318.097618,...,1.000000e+11,0.0,-1.000000e+11,0.0,1.908923,-0.416426,1.862949,2.000000e+11,2.000000e+11,0.0
3,21.346263,-56.574344,-816.973773,1292.712541,1000.001625,0.0,2.0,0.0,60.467506,310.487182,...,1.000000e+11,0.0,-1.000000e+11,0.0,1.908923,-0.416426,1.862949,2.000000e+11,2.000000e+11,0.0
4,-52.422875,-269.436745,426.095410,1121.115927,999.999653,0.0,2.0,0.0,274.489194,-486.478942,...,1.000000e+11,0.0,-1.000000e+11,0.0,1.908923,-0.416426,1.862949,2.000000e+11,2.000000e+11,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-10.481850,-31.282357,560.815533,1146.995835,999.998464,0.0,2.0,0.0,32.991742,225.074024,...,1.000000e+11,0.0,-1.000000e+11,0.0,2.581706,2.480493,0.715792,2.000000e+11,2.000000e+11,0.0
996,56.623889,-110.486930,-538.303863,1142.447011,1000.000248,0.0,2.0,0.0,124.151627,-9.990811,...,1.000000e+11,0.0,-1.000000e+11,0.0,2.581706,2.480493,0.715792,2.000000e+11,2.000000e+11,0.0
997,-168.284879,-60.692188,396.536538,1090.525095,1000.000208,0.0,2.0,0.0,178.894780,-70.905446,...,1.000000e+11,0.0,-1.000000e+11,0.0,2.581706,2.480493,0.715792,2.000000e+11,2.000000e+11,0.0
998,7.044522,33.123104,1325.310963,1660.602167,1000.001822,0.0,2.0,0.0,33.863923,-4.094953,...,1.000000e+11,0.0,-1.000000e+11,0.0,2.581706,2.480493,0.715792,2.000000e+11,2.000000e+11,0.0


In [54]:
##-- Plot variables for mass points together 
##-- All distributions together

##-- Parameters 
##################
ol = "/data_CMS/cms/portales/HHresonant_lhe/plots" ##-- Output location for plots 
gridpack_Type = "VBF_Spin-2"
legendOutside = 1 ##-- Place legend outside of plot axes 
lessPoints = 0 ##-- Only plot a few mass points 
##################

##-- Define nbins, xmin, xmax, units for variables 
def GetVarParams(v):
    VarDict = {
        "Lead_H_pt" : [150, 0, 1500, "GeV"],
        "Sublead_H_pt" : [150, 0, 1500, "GeV"],
        "X_spin" : [4, 0, 4, "unitless"],
        "X_m" : [100, 0, 3100, "GeV"],
        "X_pz" : [60, -3000, 3000, "GeV"],
        "X_eta" : [14,-7,7,"unitless"],
        "absX_pz" : [30, 0, 3000, "GeV"],
        "DR_HH" : [20, 0, 7, "unitless"],
        "pdgIds" : [40, 0, 40, "unitless"],
        "Lead_H_eta" : [14, -7, 7, "radians"],
        "Lead_H_phi" : [14, -7, 7, "radians"],
        "Sublead_H_eta" : [14, -7, 7, "radians"],
        "Sublead_H_phi" : [14, -7, 7, "radians"],
        "HH_Deta" : [50, -7, 7, "radians"] ,
        "HH_Dphi" : [50, -7, 7, "radians"],
        "absHH_Deta" : [50, 0, 7, "radians"] ,
        "absHH_Dphi" : [50, 0, 7, "radians"],        
        "X_e" : [50, 0, 5000, "GeV"],
        "X_pt" : [50, 0, 1500, "GeV"],
        "Lead_g_pz" : [60, -3000, 3000, "GeV"],
        "Lead_g_pt" : [10, 0, 100, "GeV"],
        "Sublead_g_pz" : [60, -3000, 3000, "GeV"],
        "Sublead_g_pt" : [10, 0, 100, "GeV"],
        "Lead_g_px" : [60, -3000, 3000, "GeV"],
        "Sublead_g_px" : [60, -3000, 3000, "GeV"],
        "Lead_g_py" : [60, -3000, 3000, "GeV"],
        "Sublead_g_py" : [60, -3000, 3000, "GeV"],
        "G_pz_diff" : [50, 0, 5000, "GeV"],
        "G_pz_sum" : [60, -4500, 4500, "GeV"],
        "G_E_sum" : [50, 0, 5000, "GeV"],
    }
    return VarDict[v]

##-- Get dataframe files 
df_names_ = []
prefix = "lhe_production/Dataframes/%s"%(gridpack_Type)
df_names_ = ["%s/%s"%(prefix, file) for file in os.listdir(prefix)]

##-- Order by mass for plotting order purposes  
df_names = sorted(df_names_, key = lambda x: (float(x.split('/')[-1].split('.')[0])))

##-- Define Validation Variables based on gridpack type 
GF_Vars = ["pdgIds", "X_m", "G_E_sum", "Lead_H_pt", "Sublead_H_pt", "absHH_Deta", "absHH_Dphi"]
VBF_Vars = ["pdgIds", "DR_HH","X_m", "X_pt", "X_eta","Lead_H_pt", "Lead_H_eta","Sublead_H_pt", "Sublead_H_eta", "absHH_Deta", "absHH_Dphi"]
ValVarsDict = {
    "GF_Spin-0" : GF_Vars, 
    "GF_Spin-2" : GF_Vars,
    "VBF_Spin-0" : VBF_Vars,
    "VBF_Spin-2" : VBF_Vars
}

variables = ValVarsDict[gridpack_Type]

##-- To hold values for avg +/- stdev vs. Mass plots
masses = []
N_Gluons_vals, N_Higgs_vals = [], [] 
N_Events = 10000 ##-- This is assumed based on LHE file production 

##-- Initialize mean and stdev lists to plot 
for var in variables:
    exec("%s_mean = []"%(var))
    exec("%s_stdev = []"%(var))

##-- For each variable, plot all mass point distributions and save mean and stdev values for var vs. mass point plots 
for v in variables:
    print("On variable:",v)
    
    ##-- Get variable dependent parameters 
    nbins, xmin, xmax, unit = GetVarParams(v)
    allBins = np.linspace(xmin, xmax, nbins + 1)
    
    ##-- Setup Colors 
    cm = plt.get_cmap('jet') ##-- Choose nice color scheme: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    NUM_COLORS = len(df_names)
    fig, ax = plt.subplots()
    colors = [cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)]
    colorsDoubled = [] ##-- to add error bars and hist without incrementing color 
    for ic, c in enumerate(colors):
        colorsDoubled.append(c) ##-- The purpose of doubling colors in the cycle is because both a hist and errorbar are being plotted per mass point. Want same color for both 
        colorsDoubled.append(c)
    ax.set_prop_cycle(color = colorsDoubled)
    
    ##-- For each dataframe (one per LHE file)
    for i, df_name in enumerate(df_names):
        
        ##-- Dataframe info 
        endPath = df_name.split('/')[-1]
        if(lessPoints):
            if(endPath != "250.p" and endPath != "1000.p" and endPath != "3000.p"): continue
#         colors.append(color)
        df = pickle.load( open( df_name, "rb" ) ) ##-- Load dataframe 
        plotLabel = df_name.split('/')[-1]
        plotLabel = plotLabel.replace(".p","")  
        masses.append(plotLabel)
        
        ##-- Create numpy histogram normalized to integral = 1 
        
        ##-- Special cases 
        if(v=='pdgIds' or v =='N_Gluons' or v == 'N_Higgs'):
            allVals = []
            for vals in df['pdgIds']:
                for val in vals:
                    allVals.append(val)
            nonNormyvals, binedges = np.histogram(allVals, bins=nbins, range=(xmin,xmax))
            yvals, binedges = np.histogram(allVals, bins=nbins, range=(xmin,xmax),density=False)     
        elif(v == "G_pz_diff"):
            v1 = df["Lead_g_pz"]
            v2 = df["Sublead_g_pz"]
            all_df_vals = np.subtract(v1, v2)
            nonNormyvals, binedges = np.histogram(all_df_vals, bins = nbins, range = (xmin, xmax))
            yvals, binedges = np.histogram(all_df_vals, bins=nbins, range=(xmin,xmax),density=True)  
        elif(v == "G_pz_sum"):
            v1 = df["Lead_g_pz"]
            v2 = df["Sublead_g_pz"]
            all_df_vals = np.add(v1, v2)
            nonNormyvals, binedges = np.histogram(all_df_vals, bins = nbins, range = (xmin, xmax))
            yvals, binedges = np.histogram(all_df_vals, bins=nbins, range=(xmin,xmax),density=True)
        elif(v == "G_E_sum"):
            v1 = df["Lead_g_e"]
            v2 = df["Sublead_g_e"]
            all_df_vals = np.add(v1, v2)
            nonNormyvals, binedges = np.histogram(all_df_vals, bins = nbins, range = (xmin, xmax))
            yvals, binedges = np.histogram(all_df_vals, bins=nbins, range=(xmin,xmax),density=True)            
        elif(v == "G_X_pz_diff"):
            v1 = df["Lead_g_pz"]
            v2 = df["Sublead_g_pz"]
            X_pz = df["X_pz"]
            G_pz_sum = np.add(v1, v2)
            all_df_vals = np.subtract(G_pz_sum, X_pz)
            nonNormyvals, binedges = np.histogram(all_df_vals, bins = nbins, range = (xmin, xmax))
            yvals, binedges = np.histogram(all_df_vals, bins=nbins, range=(xmin,xmax),density=True)            
            
        ##-- Non-Special cases 
        else:
            if("abs" in v):
                v_no_abs = v[:].replace("abs", "")
                all_df_vals = all_df_vals = abs(df[v_no_abs])
            else:
                v_no_abs = v[:].replace("abs", "")
                all_df_vals = df[v_no_abs]
            if("abs" in v):
                nonNormyvals, binedges = np.histogram(all_df_vals, bins = nbins, range = (xmin, xmax))
                yvals, binedges = np.histogram(all_df_vals, bins=nbins, range=(xmin,xmax),density=True)                
            else:
                nonNormyvals, binedges = np.histogram(all_df_vals, bins = nbins, range = (xmin, xmax))
                yvals, binedges = np.histogram(all_df_vals, bins=nbins, range=(xmin,xmax),density=True)                

        ##-- Save mean and stdev of histogram (except for pdgIds variable since it's a list per event)
        if(v != "pdgIds"):
            mean, stdev = np.average(all_df_vals), np.std(all_df_vals)
            exec("%s_mean.append(mean)"%(v))
            exec("%s_stdev.append(stdev)"%(v))
        
        ##-- Get Poissonian bin uncertainties and scale to normalized bin height
        nonNormYerrors = np.sqrt(nonNormyvals)
        relErrors = np.divide(nonNormYerrors,nonNormyvals, out=np.zeros_like(nonNormYerrors),where=nonNormyvals!=0) 
        errors = yvals * relErrors
        bincenters = 0.5*(binedges[1:]+binedges[:-1])
        width = float(binedges[1])-float(binedges[0])
        
        ##-- Save number of gluons and higgs bosons per event 
        ##-- Note that for VBF pdgId == 21 for incoming / outgoing quarks
        if(v=='pdgIds'):
            for ic,center in enumerate(bincenters):
                val = yvals[ic]
                if(val!=0): 
                    pdgId = int(center - width / 2)
#                     print("pdgId:",pdgId,": ",val)
                    if(pdgId == 21):
                        N_Gluons_vals.append(val)
                    elif(pdgId == 25):
                        N_Higgs_vals.append(val)
                        
        ##-- Define x errors as bin width / 2 for visual purposes 
        xerrors = []
        for bin_i in range(0,nbins): xerrors.append(width/2)
        xerrors_a = np.array(xerrors) # xerrors array 
        binedges = np.delete(binedges,0,0)  
        
        ##-- Plot normalized histogram 
        plt.hist(bincenters,
                 weights = yvals,
                 histtype = 'stepfilled',
                 edgecolor = 'black',
                 bins = allBins,
                 label=plotLabel,
                 zorder = i
#                  zorder = float(float(NUM_COLORS) - float(i)) ##-- For opposite mass point z-ordering in plot
        )           
        
        ##-- Add poissonian bin uncertainties 
        plt.errorbar(
                    bincenters,
                    yvals,
                    yerr=errors,
                    ecolor = 'black',
                    linestyle = '',
                    fmt = 'none',
#                     zorder = float(float(NUM_COLORS) - float(i)) ##-- For opposite mass point z-ordering in plot
                    zorder = i + 0.5
        )        
        
    ##-- Decorate plot 
    plt.ylabel("Entries [A.U.]")
    if(v == "Lead_H_pt" or v == "Sublead_H_pt"):
        plt.title("Higgs pT")
        xlabel = "%s [%s]"%("Higgs pT",unit)
    else:
        plt.title(v)
        xlabel = "%s [%s]"%(v,unit)
    plt.xlabel(xlabel)
    plt.tick_params(direction='in')
    ax.yaxis.set_ticks_position('both')
    ax.xaxis.set_ticks_position('both') 
    if(v == "DR_HH" or v == "absHH_Deta"): plt.yscale('log')
    if(legendOutside):
        plt.legend(title = "Mass [GeV]", bbox_to_anchor=(1.01, 1.05), loc='upper left')
        fig.set_size_inches(13, 6)
    else:
        leg = plt.legend(title = "Mass [GeV]")
        fig.set_size_inches(10, 10)
    fig.tight_layout()

    ##-- Save plot to output location as png and pdf assuming gridpack_Type directory exists in output location 
    outName1 = "%s/%s/%s.png"%(ol,gridpack_Type,v)
    outName2 = "%s/%s/%s.pdf"%(ol,gridpack_Type,v)
    plt.savefig(outName1)
    plt.savefig(outName2)
    plt.close()         
            
print("DONE")


On variable: pdgIds
On variable: DR_HH
On variable: X_m
On variable: X_pt
On variable: X_eta
On variable: Lead_H_pt
On variable: Lead_H_eta
On variable: Sublead_H_pt
On variable: Sublead_H_eta
On variable: absHH_Deta
On variable: absHH_Dphi
DONE


In [55]:
##-- Produce variable average vs. Mass point version of plots

# ##-- Plot number of particles per event 
if("pdgIds" in variables):
#     N_Gluons_vals_perE = [val/N_Events for val in N_Gluons_vals] 
    N_Higgs_vals_perE = [val/N_Events for val in N_Higgs_vals]  

#     mass_G_pairs = sorted(list(zip(masses, N_Gluons_vals_perE)), key = lambda x: (float(x[0])))
    mass_H_pairs = sorted(list(zip(masses, N_Higgs_vals_perE)), key = lambda x: (float(x[0])))

    orderedMasses = [i[0] for i in mass_H_pairs]
#     GluonsPerEvent_values = [i[1] for i in mass_G_pairs]
    HiggsPerEvent_values = [i[1] for i in mass_H_pairs]

    x_pos = [i for i, _ in enumerate(orderedMasses)]
    y_pos = np.arange(len(orderedMasses))

    ##-- Particles Per Event vs. Mass plots 
#     for p in ["Gluons", "Higgs"]:
    for p in ["Higgs"]:
        fig, ax = plt.subplots()
        fig.set_size_inches(14, 7)
        vals = eval("%sPerEvent_values"%(p))
        plt.bar(y_pos, vals, align='center', color = 'dodgerblue', edgecolor = 'black')
        plt.xticks(y_pos, orderedMasses, fontsize = 12)
        plt.xlabel('Resonant Mass [GeV]', fontsize = 20)
        plt.yticks([0, 1, 2], ["0","1","2"], fontsize = 20)
        plt.title("%s Per Event"%(p), fontsize = 35)
        fig.tight_layout()
        outName1 = "%s/%s/%s.png"%(ol,gridpack_Type,"%s_Per_Event"%(p))
        outName2 = "%s/%s/%s.pdf"%(ol,gridpack_Type,"%s_Per_Event"%(p))
        print("Saving:",outName1)
        print("Saving:",outName2)
        plt.savefig(outName1)
        plt.savefig(outName2)
        plt.close()

##-- Plot bar graphs - One bin per mass point. X axis does not scale with variable
print("Plotting Bar Graphs:")
for v in variables:
    if(v == "pdgIds"): continue 
    unit = GetVarParams(v)[3]
    print("%s vs. Mass"%(v))

    ##-- Get variable mean and stdev values per mass point 
    exec("mean_vals = %s_mean"%(v))
    exec("stdev_vals = %s_stdev"%(v))

    ##-- Sort values by increasing resonant mass value 
    exec("massMeanPairs_Sorted = sorted(list(zip(masses, mean_vals)), key = lambda x: (float(x[0])))")
    exec("massStdevPairs_Sorted = sorted(list(zip(masses, stdev_vals)), key = lambda x: (float(x[0])))")
    orderedMasses = [i[0] for i in massMeanPairs_Sorted]
    orderedMeans = [i[1] for i in massMeanPairs_Sorted]
    orderedStdevs = [i[1] for i in massStdevPairs_Sorted] 
    orderedStddevs_oneSide = [i / 2. for i in orderedStdevs] ##-- Divide by 2 to define symmetric uncertainty
    x_pos = [i for i, _ in enumerate(orderedMasses)]
    y_pos = np.arange(len(orderedMasses))   
    
    ##-- Prepare Plot 
    fig, ax = plt.subplots()
    fig.set_size_inches(14, 7) ##-- Wide because of the many resonant mass points 
    plt.bar(y_pos, orderedMeans, color = 'dodgerblue', align='center', edgecolor = 'black') 
    plt.errorbar(y_pos, orderedMeans, yerr=orderedStddevs_oneSide, ecolor = 'black', linestyle = '')
    plt.scatter(y_pos, orderedMeans, color = 'black')
    plt.xticks(y_pos, orderedMasses, fontsize = 10)
    plt.xlabel('Resonant Mass [GeV]', fontsize = 20)
    plt.yticks(fontsize = 20)
    if(v == "X_spin"): plt.yticks([-1, 0, 1], ["-1","0","1"], fontsize = 20)
    if((v == "Lead_H_pt" or v == "Sublead_H_pt") and "GF" in gridpack_Type):
        vTitle = "Higgs pt"
    else: 
        vTitle = v
    plt.title("Average %s"%(vTitle), fontsize = 35)
    plt.ylabel("Average %s [%s]"%(vTitle, unit), fontsize = 20)
    fig.tight_layout()
    outName1 = "%s/%s/%s.png"%(ol,gridpack_Type,"BarGraph_%s_vs_mass"%(vTitle))
    outName2 = "%s/%s/%s.pdf"%(ol,gridpack_Type,"BarGraph_%s_vs_mass"%(vTitle))

    plt.savefig(outName1)
    plt.savefig(outName2)
    plt.close()    
    
print("---------------------")
print("Plotting 2d Scatters:")
##-- 2D graph: Variable vs. mass point, x axis is properly scaled to mass value
for v in variables:
    if(v == "pdgIds"): continue 
    print("%s vs. Mass"%(v))
    unit = GetVarParams(v)[3]    

    ##-- Get variable mean and stdev values per mass point 
    exec("mean_vals = %s_mean"%(v))
    exec("stdev_vals = %s_stdev"%(v))

    ##-- Sort values by increasing resonant mass value 
    exec("massMeanPairs_Sorted = sorted(list(zip(masses, mean_vals)), key = lambda x: (float(x[0])))")
    exec("massStdevPairs_Sorted = sorted(list(zip(masses, stdev_vals)), key = lambda x: (float(x[0])))")
    orderedMasses = [i[0] for i in massMeanPairs_Sorted]
    orderedMeans = [i[1] for i in massMeanPairs_Sorted]
    orderedStdevs = [i[1] for i in massStdevPairs_Sorted] 
    orderedStddevs_oneSide = [i / 2. for i in orderedStdevs] ##-- Divide by 2 to define symmetric uncertainty
    x_pos = [i for i, _ in enumerate(orderedMasses)]
    y_pos = np.arange(len(orderedMasses))   
    fig, ax = plt.subplots()
    fig.set_size_inches(14, 7)   
    orderedMasses_vals = [float(val) for val in orderedMasses]
    if(v == "X_m" or v == "G_E_sum"): 
        plt.plot([0, 1], [0, 1] , linestyle = "--", transform=ax.transAxes)
        plt.xlim(0, 3500)
        plt.ylim(0, 3500)        
    plt.scatter(orderedMasses_vals, orderedMeans, color = 'black')
    plt.errorbar(orderedMasses_vals, orderedMeans, yerr=orderedStddevs_oneSide, ecolor = 'black', linestyle = '')
    if(v == "Lead_H_pt" or v == "Sublead_H_pt"):
        vTitle = "Higgs pt"
        ax.plot([0, 3100], [0, 1550], ls="--", c=".3")
    else: 
        vTitle = v[:]
    plt.title("Average %s"%(vTitle), fontsize = 35)
    plt.ylabel("Average %s [%s]"%(vTitle, unit), fontsize = 20)
    plt.xlabel('Resonant Mass [GeV]', fontsize = 20)
    if(v != "X_m" and v != "G_E_sum"): 
        plt.xlim(0, 3100)
    fig.tight_layout()
    
    outName1 = "%s/%s/%s.png"%(ol,gridpack_Type,"2dGraph_%s_vs_mass"%(vTitle))
    outName2 = "%s/%s/%s.pdf"%(ol,gridpack_Type,"2dGraph_%s_vs_mass"%(vTitle))

    plt.savefig(outName1)
    plt.savefig(outName2)
    plt.close()     
    
print("DONE")

Saving: /data_CMS/cms/portales/HHresonant_lhe/plots/VBF_Spin-2/Higgs_Per_Event.png
Saving: /data_CMS/cms/portales/HHresonant_lhe/plots/VBF_Spin-2/Higgs_Per_Event.pdf
Plotting Bar Graphs:
DR_HH vs. Mass
X_m vs. Mass
X_pt vs. Mass
X_eta vs. Mass
Lead_H_pt vs. Mass
Lead_H_eta vs. Mass
Sublead_H_pt vs. Mass
Sublead_H_eta vs. Mass
absHH_Deta vs. Mass
absHH_Dphi vs. Mass
---------------------
Plotting 2d Scatters:
DR_HH vs. Mass
X_m vs. Mass
X_pt vs. Mass
X_eta vs. Mass
Lead_H_pt vs. Mass
Lead_H_eta vs. Mass
Sublead_H_pt vs. Mass
Sublead_H_eta vs. Mass
absHH_Deta vs. Mass
absHH_Dphi vs. Mass
DONE
